In [1]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.datasets import make_classification
import onnxruntime as rt
import onnx
from skl2onnx.common.data_types import FloatTensorType
from skl2onnx import to_onnx
from sklearn.feature_selection import VarianceThreshold
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from skl2onnx import convert_sklearn
from sklearn.metrics import confusion_matrix
from sklearn.utils import resample
from model1_preprocess import resample_age, resample_gender, reweigh_address, change_labels

import pandas as pd
import numpy as np

In [3]:
def modify_data(df, age_sampling_factor, gender_sampling_factor, label_flip_percentage):
    df = resample_age(df, age_sampling_factor)
    df = resample_gender(df, gender_sampling_factor)
    df = change_labels(df, label_flip_percentage)
    return df

In [4]:
file_path = 'enter/path/to/file/here.csv' # Change to your file path
df = pd.read_csv(file_path)

In [5]:
data = modify_data(df, 2.5, 2.5, 0.02)

# Let's specify the features and the target
y = data['checked']
X = data.drop(['checked', 'Ja', 'Nee'], axis=1)
X = X.astype(np.float32)

C:\Users\vanes\AppData\Local\Temp\ipykernel_17288\848276160.py:40: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  selection['checked'] = selection['checked'].replace([0,1], [1,0])


In [6]:
from sklearn.linear_model import LinearRegression


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
sample_weights = reweigh_address(X_train)

# Preprocessing
numeric_features = list(range(X.shape[1]))  # assuming all features are numeric in make_classification
numeric_transformer = StandardScaler()

# Combine preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
    ]
)

# Pipelines for both models
linreg_pipeline = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("classifier", LinearRegression())
])

# Train both models on full training data
linreg_pipeline.fit(X_train, y_train, classifier__sample_weight=sample_weights)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9,
                                                   10, 11, 12, 13, 14, 15, 16,
                                                   17, 18, 19, 20, 21, 22, 23,
                                                   24, 25, 26, 27, 28, 29, ...])])),
                ('classifier', LinearRegression())])

In [7]:
def pred_to_binary(predictions, threshold=0.5):
    """Converts risk scores to binary values."""
    return (predictions >= threshold).astype(int)

In [8]:
# Let's convert the model to ONNX
onnx_model = convert_sklearn(
    linreg_pipeline, initial_types=[('X', FloatTensorType((None, X.shape[1])))],
    target_opset=12)

# How to get metrics such as accuracy
sess = rt.InferenceSession(onnx_model.SerializeToString())
y_pred_onnx =  sess.run(None, {'X': X_test.values.astype(np.float32)})
y_pred_onnx_binary = pred_to_binary(y_pred_onnx[0]) # Do not forget
y_test = y_test.astype(int) # Do not forget
accuracy_onnx_model = accuracy_score(y_test, y_pred_onnx_binary)

print('Accuracy of the ONNX model: ', accuracy_onnx_model)

# Let's save the model
onnx.save(onnx_model, "model/model_1.onnx")

Accuracy of the ONNX model:  0.8803613385286692
